<a href="https://colab.research.google.com/github/tomonari-masada/course2021-sml/blob/main/06_linear_regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実データの単回帰分析

## description vs prediction 

今回は、単回帰をpredictionにではなくdescriptionに使う。

* description＝与えられたデータを理解する

* prediction＝未知の入力について出力を求めたい（機械学習の主戦場）

* explanation＝因果関係の理解まで含む

https://www.slideshare.net/gshmueli/to-explain-to-predict-or-to-describe


# 今回参考にした本

Laura Igual, Santi Seguí.<br/>
Introduction to Data Science: A Python Approach to Concepts, Techniques and Applications. Springer. 2017.

https://link.springer.com/book/10.1007%2F978-3-319-50017-1

# 海氷域面積と気候変動
北半球と南半球の海氷域面積を分析し、面積が減少傾向にあるのかどうかを調べてみる。（上の本の6.2.1節より。）

データは、下記の場所から取得。

https://nsidc.org/data/seaice_index/archives


「data」タブをクリックし、以下の2つのファイルをダウンロードしておく。
*   [北極域の海氷域面積データ](ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/daily/data/N_seaice_extent_daily_v3.0.csv)
*   [南極域の海氷域面積データ](ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/daily/data/S_seaice_extent_daily_v3.0.csv)

<br/>
参考：半旬ごとにまとめたデータなら、気象庁の下記のページにある。

https://www.data.jma.go.jp/gmd/kaiyou/db/seaice/global/globe_area.html


### 1) データファイルを読み込んで前処理をする

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# 北半球のデータを扱う（南半球のデータは自習）
n_ice = pd.read_csv('N_seaice_extent_daily_v3.0.csv')
print('shape:', n_ice.shape)

In [ ]:
n_ice.head()

In [ ]:
n_ice.tail()

毎日のデータがあるわけではないことが分かる。

（例：1978年10月27日のデータはない。）

In [ ]:
# 列名の文字列に空白がある
n_ice.columns

In [ ]:
# 列名の空白を除去する
n_ice.columns = n_ice.columns.str.strip()
n_ice.columns

In [ ]:
# 必要な列だけ残す
n_ice2 = n_ice[['Year', 'Month', 'Day', 'Extent']]

In [ ]:
# 最初の行はデータのフォーマットの説明のようだ
n_ice2.head()

In [ ]:
# だから、最初の行は削除する
n_ice3 = n_ice2.iloc[1:,]
n_ice3.head()

In [ ]:
# 1978年はデータが完全でないので、削除する
n_ice4 = n_ice3.loc[n_ice3['Year'] != '1978']
n_ice4.head()

In [ ]:
n_ice4.tail()

In [ ]:
n_ice4.info()

In [ ]:
# 海氷域面積を浮動小数点数に変換する
date_cols = ['Year', 'Month', 'Day']
col_as_float = n_ice4['Extent'].astype(float)
n_ice = pd.concat([n_ice4[date_cols], col_as_float], axis=1)
n_ice.head()

In [ ]:
# 行のindexを打ち直す
n_ice.reset_index(drop=True, inplace=True)
n_ice.head()

In [ ]:
# おかしな値がないかチェックする
# （欠測値が特殊な値で表されていたりするデータもあるため。）
n_ice.describe()['Extent']

### 2) 一ヶ月ごとにデータをまとめて、平均をとる

In [ ]:
n_ice_grouped = n_ice.groupby(['Year', 'Month'], as_index=False)
n_ice_monthly = n_ice_grouped.mean()[['Year', 'Month', 'Extent']]
n_ice_monthly.head()

In [ ]:
# 変数名を付け直す
n_ice = n_ice_monthly

In [ ]:
# 可視化の準備
import numpy as np
import matplotlib.pylab as plt

横軸は年にし、同じ年のデータは縦方向にまとめてプロットしてみる。

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.xticks(rotation=90, fontsize=15)
plt.yticks(fontsize=15) 
plt.rc('font', size=20) 
plt.rc('figure', figsize=(16, 7))

x = n_ice.Year
y = n_ice.Extent
plt.scatter(x, y, color='red')
plt.xlabel('Year')
plt.ylabel('Extent')

どうやら、一年の間に、かなり値が変動するらしい。

このままだと、一年の間の変化が目立ち過ぎて、全体的な変化が分かりにくい。

### 3) 一年間の範囲内での変化のパターンを可視化する

In [ ]:
# seabornを使う準備
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

In [ ]:
# seabornの散布図で、年の違いを横断して、各月ごとに面積データを可視化する。
# つまり、面積データに季節性があるかどうかを確認する。
sns.set(rc={'figure.figsize':(16,12)})
sns.scatterplot(x="Month", y="Extent", hue="Year", data=n_ice)
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig("IceExtentByMonth.png", bbox_inches='tight')

一年間の間で、はっきりした周期的な変化パターンがあることがわかる。

つまり、「この月は、だいたいこのぐらい値」という値があることが分かる。

### 4) 一年間の範囲内に見られる変化パターンを取り除く

In [ ]:
# 各月のデータの、すべての年にわたる平均と標準偏差を求める。
grouped = n_ice.groupby('Month')
month_means = grouped.Extent.mean()
month_stds = grouped.Extent.std()
print('Means:', month_means)
print()
print('Standard deviations:',month_stds)

In [ ]:
# 試みに、ヒストグラムを月別に描いてみる。
n_ice.Extent.hist(by=n_ice_monthly.Month)

In [ ]:
# 月ごとにz値へ変換する。（よく使うデータ標準化の手法。）
for i in n_ice.Month.unique():
  idx = n_ice.Month == i
  z_values = (n_ice.loc[idx, 'Extent'] - month_means[i]) / month_stds[i]
  n_ice.loc[idx, 'Extent'] = z_values

In [ ]:
# z値に変換したので、改めて、年の違いを横断して、各月ごとに面積データを可視化する。
sns.set(rc={'figure.figsize':(16,12)})
sns.scatterplot(x="Month", y="Extent", hue="Year", data=n_ice)
plt.legend(bbox_to_anchor=(1, 1))
plt.savefig("IceExtentZscoreByMonth.png", bbox_inches='tight')

### 5) 1979年〜2021年の全範囲での変化の様子を見る

各月の個性は、すでに消した。

その上で、1979年から2021年まで、全範囲での面積の変化の傾向を見る。

In [ ]:
# seabornの単回帰のプロットを利用する。
n_ice.Year = n_ice.Year.astype(int) # Yearを数値化しておく
sns.lmplot("Year", "Extent", n_ice, aspect=2);
plt.savefig("IceExtentAllMonthsByYearlmplot.png", bbox_inches='tight')

*   直線でfittingしても、悪くなさそう。
*   そして、面積減少の傾向が見て取れる。



In [ ]:
# ピアソンの相関係数と、無相関の検定のp値を求める。
import scipy.stats
scipy.stats.pearsonr(n_ice.Year.values, n_ice.Extent.values)



---



### 6) sklearnの線形回帰を使った分析

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

x = n_ice[['Year']]
y = n_ice[['Extent']]

reg.fit(x, y)

print('Coefficients:', reg.coef_)
print('Intercept:', reg.intercept_)

フィッティングの評価

*   mean squared error ($MSE$)
*   the coefficient of determination ($R^2$)

$R^2$は$(1 - \textbf{u}/\textbf{v})$と定義される。

$\textbf{u}$ は残差の二乗和 $\sum (\textbf{y} - \hat{\textbf{y}})^2$。（$\hat{\textbf{y}}$が回帰による予測値。）

$\textbf{v}$ は、真値の平均$\bar{\textbf{y}}$ からの個々の真値$\textbf{y}$のズレの二乗の和$\sum (\textbf{y} - \bar{\textbf{y}})^2$。

$R^2$の最も良い値は1.0。これより小さくなるほど悪くなる。

In [ ]:
from sklearn import metrics

# Analysis for all months together.
x = n_ice[['Year']]
y = n_ice[['Extent']]
model = LinearRegression()
model.fit(x, y)
y_hat = model.predict(x)
plt.plot(x, y,'o', alpha=0.5)
plt.plot(x, y_hat, 'r', alpha=0.5)
plt.xlabel('year')
plt.ylabel('extent (All months)')
print("MSE:", metrics.mean_squared_error(y, y_hat))
print("R^2:", metrics.r2_score(y, y_hat))
plt.savefig("IceExtentLinearRegressionAllMonthsByYearPrediction.png", dpi=300, bbox_inches='tight')

どうやら、データは長期的なnegative trendを示している、と言ってもよさそう。




---



### 7) statsmodelsの線形回帰を使った分析

こちらのほうが、統計学的な観点からの分析に向いている。


In [ ]:
import statsmodels.api as sm

In [ ]:
x_ = sm.add_constant(x) # 切片の項を追加

In [ ]:
model = sm.OLS(y, x_)

In [ ]:
results = model.fit()

In [ ]:
results.params

In [ ]:
# statsmodelsはこのsummaryが強力。
results.summary()

# 課題6
南極海氷域について、同様に分析をしてみよう。

